In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
URL = (r'C:\Users\shaha\OneDrive\Documents\QlikView&Python_Project\A_RowData\Fac_Aute_Data.xlsx')
data = pd.read_excel(URL ) #index_col=0)

# Data Set Assessment

In [3]:
data.head(3)

,Entity Code & Name,Patient Account Number,Patient Med Rec No,Patient Class Desc,Admit Date,Discharge Date,LOS Acute,LOS ICU,Patient Age,Patient Zip,...,ICD-10-PCS Code-9,ICD-10-PCS Code-10,ICD-10-PCS Code-11,ICD-10-PCS Code-12,PT Eval,OT Eval,SLP Eval,First Therpy Date-PT,First Therpy Date-OT,First Therpy Date-SLP
0,Hospital A,A00AAAH95C,H003J8J9,Inpatient,2018-01-16,2018-01-19,3.0,NaN,51,92312.0,...,NaN,NaN,NaN,NaN,0,0,0,NaT,NaT,NaT
1,Hospital A,A00AA5HCED,H00084Z3,Inpatient,2017-11-26,2018-01-03,38.0,NaN,57,92408.0,...,NaN,NaN,NaN,NaN,1,0,0,2018-01-03,NaT,NaT
2,Hospital A,A00AA6HH5C,H0088ZX8,Inpatient,2017-12-04,2018-01-08,35.0,NaN,68,92375.0,...,NaN,NaN,NaN,NaN,0,1,1,NaT,2017-12-04,NaT


In [4]:
data.shape

(66351, 45)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66351 entries, 0 to 66350
Data columns (total 45 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Entity Code & Name         66351 non-null  object        
 1   Patient Account Number     66351 non-null  object        
 2   Patient Med Rec No         66351 non-null  object        
 3   Patient Class Desc         66351 non-null  object        
 4   Admit Date                 66351 non-null  datetime64[ns]
 5   Discharge Date             66351 non-null  datetime64[ns]
 6   LOS Acute                  64517 non-null  float64       
 7   LOS ICU                    8666 non-null   float64       
 8   Patient Age                66351 non-null  int64         
 9   Patient Zip                66336 non-null  float64       
 10  Financial Class Name       66351 non-null  object        
 11  Primary Payor Plan Name    66351 non-null  object        
 12  DRG 

In [6]:
data.describe()

,LOS Acute,LOS ICU,Patient Age,Patient Zip,DRG Code,Discharge Status Code,PT Eval,OT Eval,SLP Eval
count,64517.000000,8666.000000,66351.000000,66336.000000,66344.000000,66351.000000,66351.000000,66351.000000,66351.000000
mean,5.027450,3.705862,64.659794,88631.551073,459.520062,8.478124,0.493512,0.426278,0.122817
std,4.420572,3.987477,18.834921,10545.414516,267.893463,17.759925,0.499962,0.494539,0.328229
min,0.000000,0.000000,0.000000,656.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,2.000000,1.000000,55.000000,92312.000000,247.000000,1.000000,0.000000,0.000000,0.000000
50%,4.000000,2.000000,68.000000,92323.000000,392.000000,1.000000,0.000000,0.000000,0.000000
75%,7.000000,5.000000,78.000000,92375.000000,689.000000,6.000000,1.000000,1.000000,0.000000
max,91.000000,69.000000,112.000000,99141.000000,999.000000,95.000000,1.000000,1.000000,1.000000


###### It seems we have pt with age 0 to 112 years old. We need to check it in more details and remove pt with age 0-16


In [7]:
len(data.index)

66351

In [8]:
data = data.loc[data['Patient Age'] >=16,:]
data['Patient Age'].min()
data = data.loc[data['Patient Class Desc'] == 'Inpatient']

In [9]:
len(data.index)

61145

In [10]:
data['Entity Code & Name'].value_counts().sort_values(ascending=False)

Hospital A    33445
Hospital C    18918
Hospital B     8782
Name: Entity Code & Name, dtype: int64

In [11]:
def new_hosp_name (x):
    if x == 'Hospital A':
        x = 'Hosp A'
    elif x == 'Hospital B':
        x = 'Hosp B'
    elif x == 'Hospital C':
        x = 'Hosp C'
    return x

data['Entity Code & Name'] = data['Entity Code & Name'].apply(new_hosp_name)

In [12]:
data['Entity Code & Name'].value_counts()

Hosp A    33445
Hosp C    18918
Hosp B     8782
Name: Entity Code & Name, dtype: int64

In [13]:
# We need to get Y-Q, Y-M and Y for trend analysis;
data['DC Date_YY-Q'] = (data['Discharge Date'].dt.year).astype(str) + " Q" +(data['Discharge Date'].dt.quarter ).astype(str)

data['DC Date_YY-MM'] = (data['Discharge Date'].dt.year).astype(str) + " " +(data['Discharge Date'].dt.month ).astype(str)

data['DC Date_YY'] = (data['Discharge Date'].dt.year).astype(str)

In [14]:
# To calculate the LOS
data["Calc_LOS"] = (data['Discharge Date'] - data['Admit Date']).dt.days

In [15]:
data.columns

Index(['Entity Code & Name', 'Patient Account Number', 'Patient Med Rec No',
       'Patient Class Desc', 'Admit Date', 'Discharge Date', 'LOS Acute',
       'LOS ICU', 'Patient Age', 'Patient Zip', 'Financial Class Name',
       'Primary Payor Plan Name', 'DRG Code', 'Discharge Status Code',
       'Final Destination Name', 'ICD-10-CM Prin Diag Code',
       'ICD-10 DX Code 2', 'ICD-10 DX Code 3', 'ICD-10 DX Code 4',
       'ICD-10 DX Code 5', 'ICD-10 DX Code 6', 'ICD-10 DX Code 7',
       'ICD-10 DX Code 8', 'ICD-10 DX Code 9', 'ICD-10 DX Code 10',
       'ICD-10 DX Code 11', 'ICD-10 DX Code 12', 'Prin Proc ICD-10-PCS Code',
       'ICD-10-PCS Code-2', 'ICD-10-PCS Code-3', 'ICD-10-PCS Code-4',
       'ICD-10-PCS Code-5', 'ICD-10-PCS Code-6', 'ICD-10-PCS Code-7',
       'ICD-10-PCS Code-8', 'ICD-10-PCS Code-9', 'ICD-10-PCS Code-10',
       'ICD-10-PCS Code-11', 'ICD-10-PCS Code-12', 'PT Eval', 'OT Eval',
       'SLP  Eval', 'First Therpy Date-PT', 'First Therpy Date-OT',
       'First

# Cleaning the Name of Payors(Version1 - Assign KHRS Name)

In [16]:
data['Financial Class Name'].nunique()

8

In [17]:
data['Primary Payor Plan Name'].nunique()

178

In [18]:
Payor = data.groupby(by=['Financial Class Name', 'Primary Payor Plan Name']).agg({'Patient Account Number':'size','Patient Age':'mean'})\
.rename(columns={'Patient Account Number': 'Pt Count', 'Patient Age': 'Avd Age'}).sort_values(by=['Financial Class Name','Pt Count'], ascending=(True,False)).reset_index()
Payor

,Financial Class Name,Primary Payor Plan Name,Pt Count,Avd Age
0,Charity,CHARITY 100% APPROVED,275,50.469091
1,Charity,CHARITY,127,42.181102
2,Charity,CHARITY PENDING,107,47.214953
3,Charity,CHARITY PARTIAL APPROVAL,62,51.532258
4,Charity,BANKRUPTCY PATIENT,4,54.500000
...,...,...,...,...
173,Medicare,BX IL MEDICARE ADVANTAGE MMAI,52,64.653846
174,Medicare,CIGNA HEALTHSPRING M,17,69.352941
175,Medicare,WELLCARE MEDICARE ADVANTAGE M,13,71.769231
176,Medicare,MEDICARE HUMANA GOLD CHOICE PFFS M,7,76.857143


In [19]:
Payor.to_excel(r'C:\Users\shaha\OneDrive\Documents\QlikView&Python_Project\Python Project\payor_list.xlsx', index=False)

In [20]:
KHRS_Payor = pd.read_excel(r'C:\Users\shaha\OneDrive\Documents\QlikView&Python_Project\Python Project\KHRS_Payor.xlsx', index=False).reset_index(drop=True)

In [21]:
KHRS_Payor['Payor_key_Crosswalk'] = KHRS_Payor['Financial Class Name']+'|'+KHRS_Payor['Primary Payor Plan Name']

In [22]:
KHRS_Payor.drop(['Financial Class Name', 'Primary Payor Plan Name', 'Pt Count', 'Avd Age'], axis =1)

,KHRS_Payor,Payor_key_Crosswalk
0,Other,Charity|BANKRUPTCY PATIENT
1,Other,Charity|CHARITY
2,Other,Charity|CHARITY 100% APPROVED
3,Other,Charity|CHARITY PARTIAL APPROVAL
4,Other,Charity|CHARITY PENDING
...,...,...
175,MCR MC,Medicare|MEDICARE RR PALMETTO GBA
176,MCR MC,Medicare|SIERRA HEALTH M
177,MCR MC,Medicare|UNITED HEALTHCARE MEDICARE ADV M
178,MCR MC,Medicare|WELLCARE MEDICARE ADVANTAGE M


In [23]:
# KHRS_Payor_Crosswalk = pd.Series(KHRS_Payor['KHRS_Payor'].values,index = KHRS_Payor['Payor_key_Crosswalk']).to_dict()
KHRS_Payor_Crosswalk_Dic = KHRS_Payor.set_index('Payor_key_Crosswalk')['KHRS_Payor'].to_dict()
KHRS_Payor_Crosswalk_Dic 

{'Charity|BANKRUPTCY PATIENT': 'Other',
 'Charity|CHARITY': 'Other',
 'Charity|CHARITY 100% APPROVED': 'Other',
 'Charity|CHARITY PARTIAL APPROVAL': 'Other',
 'Charity|CHARITY PENDING': 'Other',
 'Charity|CHARITY PRE APPROVAL': 'Other',
 'Commercial|AETNA INDEMNITY': 'Comm',
 'Commercial|AUTO INSURANCE': 'Comm',
 'Commercial|CIGNA LOCAL PLUS': 'Comm',
 'Commercial|COMMERCIAL': 'Comm',
 'Commercial|COMPSYCH OTHER': 'Comm',
 'Commercial|ELAP CLAIMS': 'Comm',
 'Commercial|HARBOR LIGHT HOSPICE': 'Comm',
 'Commercial|HELP LOAN PROGRAM PENDING': 'Comm',
 'Commercial|HOSPICE': 'Comm',
 'Commercial|LIABILITY LIEN MEDICARE': 'Comm',
 'Commercial|MDSAVE SPECIAL PRICING': 'Comm',
 'Commercial|MULTIPLAN LIMITED': 'Comm',
 'Commercial|PHYSICIAN MUTUAL': 'Comm',
 'Commercial|SEX CRIMES SERVICES': 'Comm',
 'Commercial|STATE FARM AUTO INSURANCE': 'Comm',
 'Commercial|THIRD PARTY COMMERCIAL': 'Comm',
 'Commercial|UHC INDEMNITY': 'Comm',
 'Commercial|WORKERS COMP CITY OF HOBART': 'Comm',
 'Commercial|WO

In [24]:
data['Payor_Key_main'] = data['Financial Class Name']+'|'+data['Primary Payor Plan Name']

In [25]:
data["KHRS_Payor"] = data['Payor_Key_main'].map(KHRS_Payor_Crosswalk_Dic )
data.drop(['Payor_Key_main'], axis = 1).columns

Index(['Entity Code & Name', 'Patient Account Number', 'Patient Med Rec No',
       'Patient Class Desc', 'Admit Date', 'Discharge Date', 'LOS Acute',
       'LOS ICU', 'Patient Age', 'Patient Zip', 'Financial Class Name',
       'Primary Payor Plan Name', 'DRG Code', 'Discharge Status Code',
       'Final Destination Name', 'ICD-10-CM Prin Diag Code',
       'ICD-10 DX Code 2', 'ICD-10 DX Code 3', 'ICD-10 DX Code 4',
       'ICD-10 DX Code 5', 'ICD-10 DX Code 6', 'ICD-10 DX Code 7',
       'ICD-10 DX Code 8', 'ICD-10 DX Code 9', 'ICD-10 DX Code 10',
       'ICD-10 DX Code 11', 'ICD-10 DX Code 12', 'Prin Proc ICD-10-PCS Code',
       'ICD-10-PCS Code-2', 'ICD-10-PCS Code-3', 'ICD-10-PCS Code-4',
       'ICD-10-PCS Code-5', 'ICD-10-PCS Code-6', 'ICD-10-PCS Code-7',
       'ICD-10-PCS Code-8', 'ICD-10-PCS Code-9', 'ICD-10-PCS Code-10',
       'ICD-10-PCS Code-11', 'ICD-10-PCS Code-12', 'PT Eval', 'OT Eval',
       'SLP  Eval', 'First Therpy Date-PT', 'First Therpy Date-OT',
       'First

# Re-Define the DC Disposition based on CMS Definition

In [26]:
dc_disp_crosswalk = pd.read_excel(r'C:\Users\shaha\OneDrive\Documents\QlikView&Python_Project\A_RowData\Crosswalks Tables.xlsx',\
                                  sheet_name = 'DC_Code_Name_CMS' )#.rename(columns={"Unnamed: 3": "KHRS_Payor_Crosswalk"})

dc_disp_crosswalk = dc_disp_crosswalk.drop(['Destination', 'Categorization (Roll Up)'], axis = 1).\
                    set_index('DC Codes')['DC Disp_Short'].to_dict()

In [27]:
data['KHRS_Dc_Disp'] = data['Discharge Status Code'].map(dc_disp_crosswalk)

In [28]:
data.columns[-2:]    # we lost almost 400 pt due to removal of pt with age 0 - 15

Index(['KHRS_Payor', 'KHRS_Dc_Disp'], dtype='object')

# Assign DRG Name, GMLOS, Product_line, Weight based on Patient's DRG

The received data contains just DRG Code and we need add associated
info's such as GMLOS, Product Line and weight for each patient's DRG. The crosswalk comes from CMS website and IBM Truven Health.

In [29]:
DRG_Data = pd.read_excel(r'C:\Users\shaha\OneDrive\Documents\QlikView&Python_Project\A_RowData\Crosswalks Tables.xlsx',\
                                  sheet_name = 'DRGs', skiprows=1)
# DRG_Data.head(4)

In [30]:
DRG_Name = DRG_Data.drop(['FY 2019 FINAL Post-Acute DRG', 'FY 2019 FINAL Special Pay DRG', 'TYPE', 'Product Line', 'DRG', 'MDC', 
                          'Weights', 'Geometric mean LOS', 'Arithmetic mean LOS'], axis=1)\
                            .set_index('MS-DRG')['MS-DRG Title'].to_dict()
data['MS-DRG Name'] = data['DRG Code'].map(DRG_Name)

In [31]:
Product_line = DRG_Data.drop(['FY 2019 FINAL Post-Acute DRG',
       'FY 2019 FINAL Special Pay DRG', 'TYPE', 'MS-DRG Title', 'DRG', 'MDC', 'Weights',
        'Geometric mean LOS', 'Arithmetic mean LOS'], axis=1).set_index('MS-DRG')['Product Line'].to_dict()

data['Product Line'] = data['DRG Code'].map(Product_line)

data.loc[data['Product Line']==" ",'Product Line'] = 'Other'

In [32]:
# # should be removed
# DRG_ProdLine = data[['DRG Code', 'Product Line']]
# saved_URL = (r'C:\Users\shaha\OneDrive\Documents\QlikView&Python_Project\Python Project\DRG-ProdLine.xlsx')
# DRG_ProdLine.to_excel (saved_URL)

In [33]:
DRG_Weight = DRG_Data.drop(['FY 2019 FINAL Post-Acute DRG','FY 2019 FINAL Special Pay DRG', 'TYPE', 'MS-DRG Title', 'Product Line',
       'DRG', 'MDC','Geometric mean LOS', 'Arithmetic mean LOS'], axis=1).set_index('MS-DRG')['Weights'].to_dict()
data['DRG_Weight'] = data['DRG Code'].map(DRG_Weight)

In [34]:
DRG_GMLOS = DRG_Data.drop(['FY 2019 FINAL Post-Acute DRG', 'FY 2019 FINAL Special Pay DRG', 'TYPE', 'MS-DRG Title', 'Product Line',
       'DRG', 'MDC', 'Weights', 'Arithmetic mean LOS'], axis=1).set_index('MS-DRG')['Geometric mean LOS'].to_dict()
data['DRG_GMLOS'] = data['DRG Code'].map(DRG_GMLOS)

In [35]:
DRG_MDC = DRG_Data.drop(['FY 2019 FINAL Post-Acute DRG','FY 2019 FINAL Special Pay DRG', 'TYPE', 'MS-DRG Title', 'Product Line',
       'DRG', 'Weights', 'Geometric mean LOS', 'Arithmetic mean LOS'], axis=1).set_index('MS-DRG')['MDC'].to_dict()
data['DRG_MDC'] = data['DRG Code'].map(DRG_MDC)

In [36]:
data['Product Line'].value_counts().sort_index()

Alcohol & Drug Abuse          816
Cardio-Vasc-Thor Surgery     4990
Cardiovascular Diseases      8610
ENT                          2418
General Medicine            21599
General Surgery              9188
Gynecology                    449
Nephrology/Urology           5038
Neuro Sciences               2340
Obstetrics Del               3813
Obstetrics ND                 136
Oncology                      887
Orthopedics                   592
Psychiatry                    262
Name: Product Line, dtype: int64

Note: in the received data set, we have 3953 Obstetrics Del & ND cases which are not a target for rehabilitation service. We need to eliminate from the data set.

In [37]:
data.shape

(61145, 57)

In [38]:
data = data.loc[((data['Product Line']!="Obstetrics Del") &(data['Product Line']!="Obstetrics ND" )&(data['Product Line']!= '' ) & (data['Product Line']!="Neonatology" )),: ]

In [51]:
data = data.loc[data['Patient Class Desc'] == 'Inpatient']
data['DRG Code'].fillna('Other')

0        743
1        834
2        299
4        470
5        853
        ... 
66344    247
66345    908
66346    177
66349    639
66350    346
Name: DRG Code, Length: 57196, dtype: object

In [52]:
data.shape

(57196, 69)

In [50]:
data['Product Line'].value_counts().sort_index()

Alcohol & Drug Abuse          816
Cardio-Vasc-Thor Surgery     4990
Cardiovascular Diseases      8610
ENT                          2418
General Medicine            21599
General Surgery              9188
Gynecology                    449
Nephrology/Urology           5038
Neuro Sciences               2340
Oncology                      887
Orthopedics                   592
Psychiatry                    262
Name: Product Line, dtype: int64

# RIC ASSIGNMENT PROCESS

We create a copy of ICD codes for Machine Learning Process. We need the "." for cleaning ICD codes for a better machine learning process. At the same time we need ICD codes withoit "." for RIC assignment.

In [43]:
ICD_List = ['ICD-10-CM Prin Diag Code', 'ICD-10 DX Code 2', 'ICD-10 DX Code 3', 'ICD-10 DX Code 4', 'ICD-10 DX Code 5', 
            'ICD-10 DX Code 6', 'ICD-10 DX Code 7', 'ICD-10 DX Code 8', 'ICD-10 DX Code 9', 'ICD-10 DX Code 10', 
            'ICD-10 DX Code 11', 'ICD-10 DX Code 12']
for x in data[ICD_List]:
        data[x+'v0'] = data[x].str.split('.').str[0]


## Cleaning ICD DX 1-12

Firstly, lets look at the ICD DX Codes

In [44]:
data.loc[:,['ICD-10-CM Prin Diag Code', 'ICD-10 DX Code 2', 'ICD-10 DX Code 3', 'ICD-10 DX Code 4', 'ICD-10 DX Code 5', 
            'ICD-10 DX Code 6', 'ICD-10 DX Code 7', 'ICD-10 DX Code 8', 'ICD-10 DX Code 9', 'ICD-10 DX Code 10', 
            'ICD-10 DX Code 11', 'ICD-10 DX Code 12']]

,ICD-10-CM Prin Diag Code,ICD-10 DX Code 2,ICD-10 DX Code 3,ICD-10 DX Code 4,ICD-10 DX Code 5,ICD-10 DX Code 6,ICD-10 DX Code 7,ICD-10 DX Code 8,ICD-10 DX Code 9,ICD-10 DX Code 10,ICD-10 DX Code 11,ICD-10 DX Code 12
0,D25.1,D26.0,E03.9,Z79.899,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,C95.00,D61.810,J15.6,K56.7,K92.1,D63.8,R16.1,K76.0,E87.6,Z87.891,R04.0,K59.00
2,I82.402,J18.9,I25.10,E78.5,I11.9,I48.0,E11.9,Z79.4,Z86.73,Z87.891,I25.2,I82.511
4,M17.12,Z96.641,G47.30,E78.1,E78.00,Z85.828,Z87.891,Z79.82,NaN,NaN,NaN,NaN
5,A41.9,I50.23,J96.91,G93.40,J18.9,I13.0,E87.1,N17.9,L03.116,L03.115,M86.9,E11.51
...,...,...,...,...,...,...,...,...,...,...,...,...
66344,I21.19,J20.9,J43.9,I71.9,N18.2,Z87.891,Z89.021,NaN,NaN,NaN,NaN,NaN
66345,N99.71,I87.1,N99.840,R33.9,R31.0,Z79.82,NaN,NaN,NaN,NaN,NaN,NaN
66346,J69.0,J96.01,J44.1,J44.0,I11.0,E78.00,K21.9,I50.9,F32.9,F41.9,M19.90,Z87.891
66349,E11.10,E11.43,E66.01,E11.65,Z79.4,I10,K31.84,Z91.19,G47.33,NaN,NaN,NaN


We can see "." in each ICD Dx code. We need to remove them from the string

In [45]:
ICD_List = ['ICD-10-CM Prin Diag Code', 'ICD-10 DX Code 2', 'ICD-10 DX Code 3', 'ICD-10 DX Code 4', 'ICD-10 DX Code 5', 
            'ICD-10 DX Code 6', 'ICD-10 DX Code 7', 'ICD-10 DX Code 8', 'ICD-10 DX Code 9', 'ICD-10 DX Code 10', 
            'ICD-10 DX Code 11', 'ICD-10 DX Code 12']
for x in data[ICD_List]:
        data[x] = data[x].str.replace(".","")
        

In [46]:
data.loc[:,['ICD-10-CM Prin Diag Code']].head(5)

,ICD-10-CM Prin Diag Code
0,D251
1,C9500
2,I82402
4,M1712
5,A419


### Now we need to Assign RIC code for each ICD Px code¶

In [53]:
ICD_PX_Crosswalk = pd.read_excel (r'C:\Users\shaha\OneDrive\Documents\QlikView&Python_Project\A_RowData\Crosswalks Tables.xlsx',\
                                  sheet_name = 'ICD Px Cross Walk')
ICD_PX_Crosswalk = ICD_PX_Crosswalk.set_index('ICD Px 10')['RIC'].to_dict()

In [54]:
original_ICD_PX_List = ['Prin Proc ICD-10-PCS Code', 'ICD-10-PCS Code-2', 'ICD-10-PCS Code-3', 'ICD-10-PCS Code-4',
       'ICD-10-PCS Code-5']
for ICD_PX in original_ICD_PX_List:
    data[ICD_PX +"PROC"] = data[ICD_PX].map(ICD_PX_Crosswalk)

In [55]:
data.columns

Index(['Entity Code & Name', 'Patient Account Number', 'Patient Med Rec No',
       'Patient Class Desc', 'Admit Date', 'Discharge Date', 'LOS Acute',
       'LOS ICU', 'Patient Age', 'Patient Zip', 'Financial Class Name',
       'Primary Payor Plan Name', 'DRG Code', 'Discharge Status Code',
       'Final Destination Name', 'ICD-10-CM Prin Diag Code',
       'ICD-10 DX Code 2', 'ICD-10 DX Code 3', 'ICD-10 DX Code 4',
       'ICD-10 DX Code 5', 'ICD-10 DX Code 6', 'ICD-10 DX Code 7',
       'ICD-10 DX Code 8', 'ICD-10 DX Code 9', 'ICD-10 DX Code 10',
       'ICD-10 DX Code 11', 'ICD-10 DX Code 12', 'Prin Proc ICD-10-PCS Code',
       'ICD-10-PCS Code-2', 'ICD-10-PCS Code-3', 'ICD-10-PCS Code-4',
       'ICD-10-PCS Code-5', 'ICD-10-PCS Code-6', 'ICD-10-PCS Code-7',
       'ICD-10-PCS Code-8', 'ICD-10-PCS Code-9', 'ICD-10-PCS Code-10',
       'ICD-10-PCS Code-11', 'ICD-10-PCS Code-12', 'PT Eval', 'OT Eval',
       'SLP  Eval', 'First Therpy Date-PT', 'First Therpy Date-OT',
       'First

In this project, I have row data of patients of 3 hospitals, including allpayors and I try to assigned new Code whcih is applicable for Rehabilitation facilities. This is not a full version of the project due to the confidentiality of the data process. The follwoing is list of functions applied in this process:
pd.read_excel()
.head()
.shape
.info()
.describe()
.loc[] and conditinal filtering
.value_counts()
.nunique()
.dt.year).astype(str) for time analysis
.groupby(by=[]).agg({'':'').rename(columns={'': ''}).sort_values(by=[], ascending=(True,False)).reset_index()
.read_excel()
.drop()
.set_index()[].to_dict()
.map()
.str.replace(".","")
User defined function and apply method
np.where(mask,1,0)
isY = lambda.... (work as countif in excel)
for loop and If statement




## We need to Mark patients with age 85+

In [56]:
# flag patients with age more than 85 years old - function and if statement
def pt_age_85 (x):
    if x >= 85:
        x = 1
    else:
        x = 0
    return x

data['Age_85>'] = data['Patient Age'].apply(pt_age_85)

In [57]:
data['MO'] = ''

In [58]:
data['Comorbid'] = (data['Age_85>'] ==1) | (data['MO'] == 1)

In [59]:
# it is the same as countif function in excel
isY = lambda x:int(x==90)
ric90 = lambda row: isY(row['Prin Proc ICD-10-PCS CodePROC']) + isY(row['ICD-10-PCS Code-2PROC'])+\
isY(row['ICD-10-PCS Code-3PROC'])+ isY(row['ICD-10-PCS Code-4PROC'])+ isY(row['ICD-10-PCS Code-5PROC'])

data['90'] = data.apply(ric90, axis = 1)

In [60]:
data['TF1'] = data['90']>1

In [61]:
data['TF2'] = (data['90']==1) & (data['Comorbid'] == True )

In [62]:
isY = lambda x:int(x==91)
ric91 = lambda row: isY(row['Prin Proc ICD-10-PCS CodePROC']) + isY(row['ICD-10-PCS Code-2PROC'])+\
isY(row['ICD-10-PCS Code-3PROC'])+ isY(row['ICD-10-PCS Code-4PROC'])+ isY(row['ICD-10-PCS Code-5PROC'])

data['91'] = data.apply(ric90, axis = 1)

In [63]:
data['TF3'] = data['91']>1

In [64]:
data['TF4'] = (data['91']==1) & (data['Comorbid'] == True )

In [65]:
def proc(data):
    if (data['Prin Proc ICD-10-PCS CodePROC'] ==10) | +(data['ICD-10-PCS Code-2PROC']==10) | \
       (data['ICD-10-PCS Code-3PROC']==10) | (data['ICD-10-PCS Code-4PROC']==10) | (data['ICD-10-PCS Code-5PROC']==10):
        return 10
    if (data['TF2'] == True) | (data['90'] > 1):
        return 90
    if (data['TF4'] == True) | (data['91'] > 1):
        return 91
    if (data['90'] == 1) | (data['91'] == 1):
        return 8
    else:
        return "NaN"

data["PROC"] = data.apply(proc, axis = 1) 

### Now we need to Assign RIC code for each ICD Dx code

We need to load the ICD Dx Crosswalk

In [67]:
ICD_DX_Crosswalk = pd.read_excel (r'C:\Users\shaha\OneDrive\Documents\QlikView&Python_Project\A_RowData\Crosswalks Tables.xlsx',\
                                  sheet_name = 'ICD Dx Cross Walk')

In [68]:
ICD_DX_Crosswalk.head(4)

,ICD 10,ICD 10 Description,RIC
0,I6000,Ntrm subarach hemorrhage from unsp carotid sip...,1
1,I6001,Ntrm subarach hemor from right carotid siphon ...,1
2,I6002,Ntrm subarach hemorrhage from left carotid sip...,1
3,I6010,Ntrm subarach hemorrhage from unsp middle cere...,1


In [69]:
#Create Dictionary from the data frame
ICD_DX_Crosswalk = ICD_DX_Crosswalk.set_index("ICD 10")['RIC'].to_dict()

In [70]:
# data["Primary DX 1_RIC"] = data["Primary DX 1"].map(ICD_DX_Crosswalk)
# data.columns

In [71]:
original_ICD_List = ['ICD-10-CM Prin Diag Code', 'ICD-10 DX Code 2', 'ICD-10 DX Code 3', 'ICD-10 DX Code 4', 'ICD-10 DX Code 5']
for ICD_Dx in original_ICD_List:
    data[ICD_Dx +"_RIC"] = data[ICD_Dx].map(ICD_DX_Crosswalk)
    

In [72]:
replaceNARIC = ['ICD-10-CM Prin Diag Code_RIC']
for a in replaceNARIC :
    data[a] = data[a].fillna('N/A')

In [73]:
# data.loc[:,['Prin Proc ICD-10-PCS Code_RIC', 'ICD-10-PCS Code-2_RIC', 'ICD-10-PCS Code-3_RIC', 'ICD-10-PCS Code-4_RIC',
#        'ICD-10-PCS Code-5_RIC']]

# RIC 99 Assignment

In [74]:
def RIC_99(data):
    if data['ICD-10 DX Code 2_RIC'] == 1:
        return 99
    if data['ICD-10 DX Code 3_RIC'] == 1:
        return 99
    if data['ICD-10 DX Code 4_RIC'] == 1:
        return 99
    if data['ICD-10 DX Code 5_RIC'] == 1:
        return 99
    else:
        return "No RIC 99"
data["RIC_99"] = data.apply(RIC_99, axis=1)

# RIC Selection -  Step 1/3

In [75]:
data['ICD-10-CM Prin Diag Code_RIC']

0        N/A
1         20
2         20
4         12
5        N/A
        ... 
66344     14
66345    N/A
66346     20
66349    N/A
66350    N/A
Name: ICD-10-CM Prin Diag Code_RIC, Length: 57196, dtype: object

In [76]:
def pre_final_ric(x):
    if x['ICD-10-CM Prin Diag Code_RIC'] == 1:
        return x['ICD-10-CM Prin Diag Code_RIC']
    if x["RIC_99"] == 99:
        return x["RIC_99"]
    if (x['PROC'] in [8,90,91,10]) & (x["RIC_99"] != 99) & (x['ICD-10-CM Prin Diag Code_RIC'] != 1):
        return x['PROC']
    if (x['PROC'] in [8,90,91,10]) & (x["RIC_99"] == 99) & (x['ICD-10-CM Prin Diag Code_RIC'] != 1) :
        return x["RIC_99"]
    if x['ICD-10-CM Prin Diag Code_RIC'] != 'NaN':
        return x['ICD-10-CM Prin Diag Code_RIC']
    else:
        return "No RIC"
    
data['RIC_S1a'] = data.apply(pre_final_ric, axis = 1)

## Asign RIC 20 for IRF DC Pt with No RIC

In [77]:
def ric_S1b(data):
    if  (data['RIC_S1a'] == 'N/A') & (data["KHRS_Dc_Disp"] =='IRF'):
        return 20
    else:
        return data['RIC_S1a']

data['RIC_S1b'] = data.apply(ric_S1b, axis = 1)

In [78]:
# data.loc[((data['RIC_S1a'] == 'N/A') & (data["KHRS_Dc_Disp"] =='IRF')),:]

# Presumptive RIC Code 

In [80]:
presum = pd.read_excel (r'C:\Users\shaha\OneDrive\Documents\QlikView&Python_Project\A_RowData\Crosswalks Tables.xlsx',\
                                  sheet_name = 'Presump_Code')

In [81]:
presum.RIC.unique()

array([ 1,  2,  3,  4,  5,  6,  7,  9, 10, 11, 13, 15, 16, 19, 20, 21],
      dtype=int64)

In [82]:
presum_dict  = presum.set_index('Presum Code')['RIC'].to_dict()

In [83]:
listOfDX_RIC = ['ICD-10 DX Code 2', 'ICD-10 DX Code 3', 'ICD-10 DX Code 4','ICD-10 DX Code 5'] 
for a in listOfDX_RIC:
    data[a+"presum"] = data[a].map(presum_dict)

In [84]:

isY = lambda x:int(x in [ 1,  2,  3,  4,  5,  6,  7,  9, 10, 11, 13, 15, 16, 19, 20, 21])
Any_Presume = lambda row: isY(row['ICD-10 DX Code 2presum']) + isY(row['ICD-10 DX Code 3presum'])+ isY(row['ICD-10 DX Code 4presum'])+ isY(row['ICD-10-PCS Code-4PROC'])+ isY(row['ICD-10 DX Code 5presum'])
data['Any Presumptive Code'] = data.apply(Any_Presume, axis = 1)


In [85]:
replaceNA = ['ICD-10 DX Code 2presum','ICD-10 DX Code 3presum', 'ICD-10 DX Code 4presum', 'ICD-10 DX Code 5presum']
for x in replaceNA:
    data[x] = data[x].fillna('N/A')

In [86]:
def presumptive_code(Y):
    if Y['ICD-10 DX Code 2presum'] != "N/A":
        return Y['ICD-10 DX Code 2presum']
    if Y['ICD-10 DX Code 2presum'] == 'N/A' and Y['ICD-10 DX Code 3presum'] != "N/A":
        return Y['ICD-10 DX Code 3presum']
    if Y['ICD-10 DX Code 2presum'] == 'N/A' and Y['ICD-10 DX Code 3presum'] == "N/A" and Y['ICD-10 DX Code 4presum'] != "N/A":
        return Y['ICD-10 DX Code 4presum']
    if Y['ICD-10 DX Code 2presum'] == 'N/A' and Y['ICD-10 DX Code 3presum'] == "N/A" and Y['ICD-10 DX Code 4presum'] == "N/A" and Y['ICD-10 DX Code 5presum'] != "N/A":
        return Y['ICD-10 DX Code 5presum']
    else:
        return np.nan

data['Presumptive'] = data.apply(presumptive_code, axis = 1)

In [87]:
def ric_s2 (data):
    if (data['Any Presumptive Code'] > 0) & ((data['RIC_S1b']==20) |(data['RIC_S1b']=='N/A')):
        return data['Presumptive']
    else:
        return data['RIC_S1b']

data['RIC_S2'] = data.apply(ric_s2, axis = 1)

In [89]:
RIC_Desc = pd.read_excel(r'C:\Users\shaha\OneDrive\Documents\QlikView&Python_Project\A_RowData\Crosswalks Tables.xlsx', sheet_name = 'RIC Desc' )

RIC_Desc

,RIC Code,RIC Description,RIC Group
0,1,Ric 1 Stroke - Primary,Neuro
1,99,Ric 1 Stroke - Secondary,Neuro
2,2,Ric 2 BI - Traumatic,Neuro
3,3,Ric 3 BI - NonTraumatic,Neuro
4,4,Ric 4 SCI - Traumatic,Neuro
5,5,Ric 5 SCI - NonTraumatic,Neuro
6,6,Ric 6 Neurological,Neuro
7,7,Ric 7 Fracture,Ortho
8,8,Ric 8 Joint Rep (Other),Ortho
9,90,Ric 8 Bilat - THR,Ortho


In [90]:
RIC_Description = RIC_Desc.set_index("RIC Code")['RIC Description'].to_dict()

In [91]:
RIC_Group =       RIC_Desc.set_index("RIC Code")['RIC Group'].to_dict()

In [92]:
data['Final_RIC_Desc'] = data['RIC_S2'].map(RIC_Description)
# np.where(inpatient['RIC_S2']!='N/A',inpatient['RIC_S2'],"No RIC Match")

In [93]:
data.loc[data['RIC_S2']=='N/A', 'Final_RIC_Desc'] = 'No RIC Match'
data.loc[data['RIC_S2']!='N/A', 'Final_RIC_Desc'] = data['Final_RIC_Desc']

In [94]:
data['Final_RIC_Group'] = data['RIC_S2'].map(RIC_Group)
# np.where(inpatient['RIC_S2']!='N/A',inpatient['RIC_S2'],"No RIC Match")

In [95]:
data.loc[data['RIC_S2']=='N/A', 'Final_RIC_Group'] = 'No RIC Match'
data.loc[data['RIC_S2']!='N/A', 'Final_RIC_Group'] = data['Final_RIC_Group']

In [96]:
data.loc[:,['Final_RIC_Desc', 'Final_RIC_Group']]

,Final_RIC_Desc,Final_RIC_Group
0,No RIC Match,No RIC Match
1,Ric 20 Miscellaneous,Medically Complex
2,Ric 20 Miscellaneous,Medically Complex
4,Ric 12 Osteoarthritis,Ortho
5,No RIC Match,No RIC Match
...,...,...
66344,Ric 14 Cardiac,Medically Complex
66345,No RIC Match,No RIC Match
66346,Ric 20 Miscellaneous,Medically Complex
66349,No RIC Match,No RIC Match


In [97]:
data['Patient Class Desc'].value_counts()

Inpatient    57196
Name: Patient Class Desc, dtype: int64

The data contains pt records for Rehab, Hospice and Surgery. We should exclude them from the data for our analysis.

In [98]:
inpatient = data.loc[data['Patient Class Desc'] == 'Inpatient']

In [99]:
# conditions= [inpatient['RIC_S2'] == 'N/A', inpatient['RIC_S2'] != 'N/A']

# choices = ['No RIC Match', inpatient['RIC_S2']]
# inpatient['Final_RIC_Desc'] = np.select(conditions, choices, default='N/A')

In [100]:
# def final_ric_group(x):
#     if x['RIC_S2'] == 'N/A':
#         return 'No RIC Match'
#     if x['RIC_S2'] in [1,99,2,3,4,5,6,19]:
#         return 'Neuro'
#     if x['RIC_S2'] in [7,8,9,90,91,12,13]:
#         return 'Ortho'    
#     if x['RIC_S2'] in [10,11,14,15,16,20,21]:
#         return 'Med Complx'       
    
# inpatient['Final_RIC_Group'] = inpatient.apply(final_ric_group, axis = 1)

In [101]:
# inpatient['Final_RIC_Desc'].head(10)

In [102]:
# # for x in data['Final RIC_S3']:
# inpatient['Final_RIC_Group2'] = inpatient['RIC_S2'].map(RIC_Group)
# # mask = inpatient['Final_RIC_Group2'] != 'NaN'
# inpatient['Final_RIC_Group3'] = np.where(inpatient['Final_RIC_Group2'] =="", "No RIC Match", inpatient['Final_RIC_Group2'])
# inpatient['Final_RIC_Group3']

In [103]:
# Assign ID for each raw
inpatient["Counter"] = range(len(inpatient))

In [104]:
#  assign count 1
inpatient['Count'] = 1

In [105]:
inpatient.shape

(57196, 103)

At the begining, we had ~66000 patient records, but at the end we have 57196 patinent records. 
THe differece is pt age 0-16, new born, DRGs relevant to OB, NewBorn, Pt at Rehab, Hospice and Surgery Center.

In [107]:
# we export inpatient data with age 17+ and exlucing OB, NewBorn for Post Acute Care Analysis!
saved_URL = (r'C:\Users\shaha\OneDrive\Documents\QlikView&Python_Project\Python Project\Inp Data - first outcome.xlsx')
inpatient.to_excel (saved_URL)